In [1]:
using Parameters, AutomotiveDrivingModels, POMDPs, AutoViz, Reel

In [2]:
include("occluded_crosswalk_env.jl")
include("helpers.jl")
include("pomdp_types.jl")
include("constant_pedestrian.jl")
include("generative_model.jl")
include("render_helpers.jl")

In [3]:
rng = MersenneTwister(1)
pomdp = OCPOMDP();



In [4]:
s0 = initial_state(pomdp, rng)
ego = s0[findfirst(s0,1)]
println("Initial scene with $(length(s0)) cars")
println("Initial ego state: $ego")

Initial scene with 1 cars
Initial ego state: Vehicle(1, VehicleState(VecSE2({5.000, 0.000}, 0.000), Frenet(RoadIndex({1, 0.100000}, {1, 1}), 5.000, 0.000, 0.000), 6.016), VehicleDef(CAR, 4.000, 1.800))


In [27]:
# Try to simulate
done = false
max_steps = 100
step = 0
rtot = 0.
s = initial_state(pomdp, rng)
rec = rec = SceneRecord(max_steps+1, pomdp.ΔT)
update!(rec, s)
while !done && step < max_steps
    sp, o, r = generate_sor(pomdp, s, OCAction(0.0), rng)
    done = isterminal(pomdp, sp)
    rtot += r
    s = sp
    update!(rec, sp)
    step += 1
end

In [30]:
size(convert_o(Vector{Float64}, o, pomdp))

(22,)

In [10]:
overlay = LidarOverlay(pomdp.sensor);

In [11]:
fps, duration, render_rec = animate_record(rec, pomdp, overlay)
film = roll(render_rec, fps = fps, duration = duration)


Reel.Frames{MIME{Symbol("image/png")}}("C:\\Users\\Maxime\\AppData\\Local\\Temp\\jl_D8C9.tmp", 0x0000000000000007, 3.5, nothing)

## Measure maximum number of pedestrians

In [7]:
max_ped = 0
n_ep = 100
for i=1:n_ep
    done = false
    max_steps = 100
    step = 0
    rtot = 0.
    s = initial_state(pomdp, rng)
    rec = rec = SceneRecord(max_steps+1, pomdp.ΔT)
    update!(rec, s)
    while !done && step < max_steps
        sp, o, r = generate_sor(pomdp, s, OCAction(0.0), rng)
        if length(sp) > max_ped
            max_ped = length(sp)
        end
        for veh in sp
            println(veh.id)
        end
        done = isterminal(pomdp, sp)
        rtot += r
        s = sp
        update!(rec, sp)
        step += 1
    end
end
max_ped -= 1
println("Maximum number of pedestrians observed over $n_ep: $max_ped")

2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


LoadError: [91mAssertionError: veh.id <= pomdp.max_ped + 1[39m